In [ ]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
from PIL import Image

Obtenemos las imágenes sobre las temáticas y las guardamos en el directorio

In [ ]:
object_types = ["mercedes", "porsche", "bmw", "audi"]
path = Path('cars')
numberOfMaxImages = 150

if not path.exists():
  path.mkdir()
  
for o in object_types:
  dest = (path/o)
  dest.mkdir(exist_ok=True)
  results = search_images_ddg(f'{o}')
  
  cont = 0
  index = 0
  while cont < numberOfMaxImages and index < len(results):
    try: 
      download_url(results[index], str(dest) + "/" + str(cont) + ".jpg", timeout=20)
      cont += 1
    except:
      pass
    finally:
      index += 1

In [ ]:
#def to_black_white(img):
#    img = Image.open(img, 'r')
#    img = img.convert('L')
#    img.save(img.filename)

#def convert_to_grayscale(path):
#    for item in path.iterdir():
#        if(item.is_file()):
#            convert_to_grayscale(item)
#        else:
#            to_black_white(item)
#
#convert_to_grayscale(path)

Limpiamos las imágenes descargadas que estén corruptas

In [ ]:
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink)

Creamos el DataLoader

In [ ]:
cars = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

Entrenamos el modelo con las imágenes

In [ ]:
cars = cars.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = cars.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner
#hide
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

Exportamos el modelo a un archivo .pkl

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf.dls.vocab